### 1. Prices

Both asset and product prices were scraped from,

`https://derivative.credit-suisse.com/ch/ch/en/detail/drop-back-certificate-s-p-500/CH1199361879/119936187`

In [ ]:
# Insert raw JSON response here

In [1]:
import json
import pandas as pd

json_str = json.dumps(asset_raw_data)
df = pd.read_json(json_str)
df = df.set_index("date")
df.head()

In [55]:
df = pd.read_csv("asset_prices_raw.csv")
df["date"] = pd.to_datetime(df["date"])
df = df.set_index("date")
df.head()

,value
date,
2018-11-13,2722.18
2018-11-14,2701.58
2018-11-15,2730.20
2018-11-16,2736.27
2018-11-19,2690.73


In [ ]:
# # For asset prices, we only want one column
# df = df.drop(['high', 'low', 'close', 'open'], axis=1)
# df.head()

In [56]:
# Filter out relevant data
start_date = pd.to_datetime('2022-07-19')
end_date = pd.to_datetime('2023-11-09')
filtered_df = df[(df.index >= start_date) & (df.index <= end_date)]

# Most recent data first
filtered_df = filtered_df.sort_index()
filtered_df

,value
date,
2022-07-19,3936.69
2022-07-20,3959.90
2022-07-21,3998.95
2022-07-22,3961.63
2022-07-25,3966.84
...,...
2023-11-02,4317.78
2023-11-03,4358.34
2023-11-06,4365.98


In [57]:
# Save file
filtered_df.to_csv("asset_prices_processed.csv")

In [37]:
# Sanity check
a = pd.read_csv("asset_prices_processed.csv")
a["date"] = pd.to_datetime(a["date"])
a = a.set_index("date")

p = pd.read_csv("product_prices_processed.csv")
p["date"] = pd.to_datetime(p["date"])
p = p.set_index("date")

# assert len(a) == len(p) + 1

In [38]:
for x, y in zip(a.index, p.index):
    if x != y:
        print(x, y)
        break

2023-02-21 00:00:00 2023-02-20 00:00:00


### 2. Daily Interest Rate

Downloaded the 2022 and 2023 daily US Treasury Rates from,

`https://home.treasury.gov/policy-issues/financing-the-government/interest-rate-statistics`

In [59]:
# Ensure these are same as above
start_date = pd.to_datetime('2022-07-19')
end_date = pd.to_datetime('2023-11-09')

In [60]:
rate_2022 = pd.read_csv("daily-treasury-rates-2022.csv")
rate_2022["Date"] = pd.to_datetime(rate_2022["Date"])
rate_2022 = rate_2022.set_index("Date")
rate_2022.head()

,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
Date,,,,,,,,,,,,,
2022-12-30,4.12,4.41,4.42,4.69,4.76,4.73,4.41,4.22,3.99,3.96,3.88,4.14,3.97
2022-12-29,4.04,4.39,4.45,4.66,4.73,4.71,4.34,4.16,3.94,3.91,3.83,4.09,3.92
2022-12-28,3.86,4.33,4.46,4.66,4.75,4.71,4.31,4.18,3.97,3.97,3.88,4.13,3.98
2022-12-27,3.87,4.32,4.46,4.66,4.76,4.75,4.32,4.17,3.94,3.93,3.84,4.10,3.93
2022-12-23,3.80,4.20,4.34,4.59,4.67,4.66,4.31,4.09,3.86,3.83,3.75,3.99,3.82


In [61]:
filtered_rate_2022 = rate_2022[rate_2022.index >= start_date]
filtered_rate_2022 = filtered_rate_2022.sort_index()
filtered_rate_2022

,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
Date,,,,,,,,,,,,,
2022-07-19,1.93,2.29,2.52,NaN,3.06,3.18,3.23,3.22,3.14,3.11,3.01,3.42,3.17
2022-07-20,1.92,2.29,2.51,NaN,3.04,3.18,3.25,3.25,3.18,3.15,3.04,3.43,3.17
2022-07-21,2.15,2.30,2.48,NaN,3.00,3.11,3.10,3.07,3.00,2.99,2.91,3.33,3.08
2022-07-22,2.15,2.29,2.49,NaN,2.97,3.01,2.98,2.93,2.87,2.85,2.77,3.23,3.00
2022-07-25,2.14,2.31,2.62,NaN,3.06,3.07,3.00,2.98,2.89,2.89,2.81,3.28,3.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,3.80,4.20,4.34,4.59,4.67,4.66,4.31,4.09,3.86,3.83,3.75,3.99,3.82
2022-12-27,3.87,4.32,4.46,4.66,4.76,4.75,4.32,4.17,3.94,3.93,3.84,4.10,3.93
2022-12-28,3.86,4.33,4.46,4.66,4.75,4.71,4.31,4.18,3.97,3.97,3.88,4.13,3.98


In [62]:
rate_2023 = pd.read_csv("daily-treasury-rates-2023.csv")
rate_2023["Date"] = pd.to_datetime(rate_2023["Date"])
rate_2023 = rate_2023.set_index("Date")
rate_2023.head()

,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
Date,,,,,,,,,,,,,
2023-11-10,5.53,5.55,5.53,5.47,5.46,5.38,5.04,4.80,4.65,4.68,4.61,4.93,4.73
2023-11-09,5.53,5.56,5.54,5.50,5.46,5.39,5.03,4.77,4.65,4.68,4.62,4.97,4.77
2023-11-08,5.52,5.54,5.54,5.49,5.46,5.34,4.93,4.65,4.51,4.54,4.49,4.82,4.64
2023-11-07,5.53,5.56,5.55,5.48,5.47,5.33,4.91,4.64,4.53,4.58,4.58,4.91,4.75
2023-11-06,5.53,5.56,5.56,5.49,5.47,5.33,4.93,4.72,4.60,4.66,4.67,5.00,4.84


In [63]:
filtered_rate_2023 = rate_2023[rate_2023.index <= end_date]
filtered_rate_2023 = filtered_rate_2023.sort_index()
filtered_rate_2023

,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
Date,,,,,,,,,,,,,
2023-01-03,4.17,4.42,4.53,4.70,4.77,4.72,4.40,4.18,3.94,3.89,3.79,4.06,3.88
2023-01-04,4.20,4.42,4.55,4.69,4.77,4.71,4.36,4.11,3.85,3.79,3.69,3.97,3.81
2023-01-05,4.30,4.55,4.66,4.75,4.81,4.78,4.45,4.18,3.90,3.82,3.71,3.96,3.78
2023-01-06,4.32,4.55,4.67,4.74,4.79,4.71,4.24,3.96,3.69,3.63,3.55,3.84,3.67
2023-01-09,4.37,4.58,4.70,4.74,4.83,4.69,4.19,3.93,3.66,3.60,3.53,3.83,3.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-03,5.53,5.56,5.53,5.50,5.45,5.29,4.83,4.62,4.49,4.55,4.57,4.93,4.77
2023-11-06,5.53,5.56,5.56,5.49,5.47,5.33,4.93,4.72,4.60,4.66,4.67,5.00,4.84
2023-11-07,5.53,5.56,5.55,5.48,5.47,5.33,4.91,4.64,4.53,4.58,4.58,4.91,4.75


In [64]:
rate_combined = pd.concat([filtered_rate_2022, filtered_rate_2023])
rate_combined

,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
Date,,,,,,,,,,,,,
2022-07-19,1.93,2.29,2.52,NaN,3.06,3.18,3.23,3.22,3.14,3.11,3.01,3.42,3.17
2022-07-20,1.92,2.29,2.51,NaN,3.04,3.18,3.25,3.25,3.18,3.15,3.04,3.43,3.17
2022-07-21,2.15,2.30,2.48,NaN,3.00,3.11,3.10,3.07,3.00,2.99,2.91,3.33,3.08
2022-07-22,2.15,2.29,2.49,NaN,2.97,3.01,2.98,2.93,2.87,2.85,2.77,3.23,3.00
2022-07-25,2.14,2.31,2.62,NaN,3.06,3.07,3.00,2.98,2.89,2.89,2.81,3.28,3.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-03,5.53,5.56,5.53,5.50,5.45,5.29,4.83,4.62,4.49,4.55,4.57,4.93,4.77
2023-11-06,5.53,5.56,5.56,5.49,5.47,5.33,4.93,4.72,4.60,4.66,4.67,5.00,4.84
2023-11-07,5.53,5.56,5.55,5.48,5.47,5.33,4.91,4.64,4.53,4.58,4.58,4.91,4.75


In [65]:
# Save file
rate_combined.to_csv("daily_interest_rates.csv")

### 3. Dividends 

We manually collected the quarterly dividend rates of the <b>SPDR S&P 500 Trust ETF</b> <i>(NYSE Symbol - SPY)</i> from,

`https://www.nasdaq.com/market-activity/etf/spy/dividend-history`

Now we have the following files,

<ol>
<li> daily_interest_rates.csv
<li> asset_prices_processed.csv
<li> product_prices_processed.csv
<li> SPY_quarterly_dividend_rates.csv
</ol>

<br>
and we can begin our product pricing simulations.